# Step 1

Use your own directory and filename

Must have column names: Latitude, Longitude

In [1]:

import pandas as pd
import numpy as np
import json
import requests
from math import radians, sqrt, sin, cos, atan2

dirpath = 'C:/Users/thuang1/Desktop/REST API/'
filename = 'XD_End.csv'



# Step 2

a. Functions

In [2]:
def geocalc(lat1, lon1, lat2, lon2):
    lat1 = radians(lat1)
    lon1 = radians(lon1)
    lat2 = radians(lat2)
    lon2 = radians(lon2)

    dlon = lon1 - lon2

    EARTH_R = 6372.8

    y = sqrt(
        (cos(lat2) * sin(dlon)) ** 2
        + (cos(lat1) * sin(lat2) - sin(lat1) * cos(lat2) * cos(dlon)) ** 2
        )
    x = sin(lat1) * sin(lat2) + cos(lat1) * cos(lat2) * cos(dlon)
    c = atan2(y, x)
    return EARTH_R * c

In [3]:
def get_MM(d, dat):
    route_des = pd.DataFrame({'route_desc':['S','C','M','P','I'], 'route_desc_order':[1,2,3,4,5]})
            
    xlong = d['geometry']['x']
    xlati = d['geometry']['y']
    
    df2 = pd.DataFrame(dat, columns=['routeId','MM','long','lati'])
    df2['route_desc']=df2.routeId.str[0]
    df2['sys_code']=df2.routeId.str[5]
    df2['route_no']=df2.routeId.str[7:10]
    df2['is_ramp'] = 1
    df2.loc[pd.isnull(df2.routeId.str[11]),'is_ramp'] = 0
    df2 = df2.join(route_des.set_index(['route_desc']), on=['route_desc'])
    df2['exception']=1
    df2.loc[(df2['sys_code']=='1') & (df2['route_no']=='080'),'exception']=0
    df2['dist']=df2.apply(lambda x: geocalc(float(x['lati']),float(x['long']),xlati,xlong), axis=1)
    df2 = df2.sort_values(by=['is_ramp','exception','route_desc_order','sys_code','route_no','dist']).reset_index(drop=True)
    routeId = df2.loc[0,'routeId']
    MM = df2.loc[0,'MM']
    return np.array([routeId, MM])

In [4]:
def get_locations(df):
    d = []
    for i in xrange(len(df)):
        x = {'geometry':{'y':df.iloc[i]['Latitude'], 'x':df.iloc[i]['Longitude']}}
        d.append(x)

    locations = json.dumps(d, ensure_ascii=False)
    return d, locations

In [5]:
def get_measure(locations, tolerance):
       

    url = 'https://gis.iowadot.gov/ramsa/rest/services/lrs/MapServer/exts/LRSServer/networkLayers/0/geometryToMeasure?'
    data = {'f':'json','locations': locations,'tolerance':str(tolerance),'inSR':'4326'}
    response = requests.post(url, data=data)
    r=response.json()

        
        
    # extract results
    resall=[]
    for i in r['locations']:
        res=[]
        if i['status']==u'esriLocatingCannotFindLocation':
            routeid="0"
            measure=0
            x=0
            y=0
            res.append([routeid, measure, x, y])
        else:

            for j in i['results']:
                routeid=j['routeId'].encode('ascii','ignore')
                measure=j['measure']
                x=j['geometry']['x']
                y=j['geometry']['y']
                res.append([routeid, measure, x, y])
        resall.append(res)
    
    

    datall=[]
    dat=[]

    for i in xrange(len(resall)):

        if resall[i][0][0]=='0':
            dat = ['0', 0]
        else:
            dat = get_MM(d[i], resall[i])
        datall.append(dat)

    datall=np.array(datall)
    
    return datall

b. Use n to slice your data into chunks with n rows in each one.

The API only returns at most 1500 responses, thus, 0<=n<=1500

In [7]:
# load your data
df = pd.read_csv(dirpath+filename)

n=500

ans=[]
for i in xrange(len(df)/n+1):
    ans.append(df[i*n:i*n+n])
    


c. Main function

In [8]:
# Main

from datetime import datetime
now1=datetime.now()

final=[]
i=0

for a in ans:
    i=i+1
    d, locations = get_locations(a)
    
    datall = get_measure(locations, 50)
        
    a['routeId']=datall[:,0]
    a['MM']=datall[:,1]
    
    final.append(a)
    
    print('Success! Progress: '+str(round(float(i)/len(ans)*100,2))+"%")

final=pd.concat(final,axis=0).reset_index(drop=True)

now2=datetime.now()

print('Runtime: '+str(now2-now1))

C:\Users\thuang1\AppData\Local\Continuum\Anaconda2\lib\site-packages\ipykernel\__main__.py:15: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
C:\Users\thuang1\AppData\Local\Continuum\Anaconda2\lib\site-packages\ipykernel\__main__.py:16: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy


Success! Progress: 5.88%
Success! Progress: 11.76%
Success! Progress: 17.65%
Success! Progress: 23.53%
Success! Progress: 29.41%
Success! Progress: 35.29%
Success! Progress: 41.18%
Success! Progress: 47.06%
Success! Progress: 52.94%
Success! Progress: 58.82%
Success! Progress: 64.71%
Success! Progress: 70.59%
Success! Progress: 76.47%
Success! Progress: 82.35%
Success! Progress: 88.24%
Success! Progress: 94.12%
Success! Progress: 100.0%
Runtime: 0:09:42.680000


d. Save the results

In [9]:
# save new file

final.to_csv(dirpath+'MM_Found_'+filename, header=True, index=False)